In [2]:
import os
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers

In [3]:
config = {
    "dataset_path": "images/",
    "validation_ratio": 0.2,
    "image_size": (300, 300),
    "batch_size": 32,
    "mode": "all",
    "loss_function": "categorical_crossentropy",
    "optimizer": "adam",
    "epochs": 3,
    "checkpoint_pattern": "checkpoint/cp-{epoch:03d}.ckpt",
    "checkpoint_dir": "checkpoint/"
}

# train/test 데이터셋 나누기

In [4]:
def image_loader(directory,
                 validation_split, 
                 image_size, 
                 batch_size,
                 subset="all"):
    
    train_ds, valid_ds = None, None
    if "all" == subset:
        subset = "train/valid"
    
    if "train" in subset:
        train_ds = image_dataset_from_directory(
            directory,
            labels='inferred',
            label_mode='categorical',
            validation_split=validation_split,
            subset="training",
            seed=123,
            image_size=image_size,
            batch_size=batch_size)
        
    if "valid" in subset:
        valid_ds = image_dataset_from_directory(
            directory,
            labels='inferred',
            label_mode='categorical',
            validation_split=validation_split,
            subset="validation",
            seed=123,
            image_size=image_size,
            batch_size=batch_size)
    
    return train_ds, valid_ds

def normalize(ds):
    normalizer = layers.experimental.preprocessing.Rescaling(1./255)
    normlized_ds = ds.map(lambda x, y: (normalizer(x), y))
    return normlized_ds

In [5]:
train_ds, valid_ds = image_loader('images/', 0.2, (300, 300), 32)

Found 45000 files belonging to 3 classes.
Using 36000 files for training.
Found 45000 files belonging to 3 classes.
Using 9000 files for validation.


In [6]:
train_ds.class_names

['exterior', 'food', 'interior']

In [7]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 300, 300, 3)
(32, 3)


In [8]:
normalized_ds = normalize(train_ds)

for image_batch, labels_batch in normalized_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    sample_label = labels_batch[31]
    sample_image = image_batch[31]
    print(sample_label)
    print(np.min(sample_image), np.max(sample_label))
    break

(32, 300, 300, 3)
(32, 3)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
0.0 1.0


# 콜백 작성하기

In [10]:
class CustomCallback(ks.callbacks.Callback):

    def __init__(self):
        self.accs = {}
        self.losses = {}

    def on_epoch_end(self, epoch, logs):
        self.losses[epoch] = logs['loss']
        print('\n==========\n')
        print('Epoch:', epoch)
        print('Training Loss:', logs['loss'])
        print('Validation Loss:', logs['val_loss'])
        print('Accuracy:', logs['acc'])

    def on_train_end(self, logs):
        import matplotlib.pyplot as plt
        plt.plot(self.losses.keys(), self.losses.values())
        plt.xlabel('Training Epochs')
        plt.ylabel('Training Loss')
        plt.ylim([0, 2]) #수렴 여부를 관찰하기 위해 좁은 뷰 설정
        plt.show()

        self.losses = {}

recorder = CustomCallback()
checkpoint_callback = ks.callbacks.ModelCheckpoint(
    config["checkpoint_pattern"], 
    monitor='val_acc', verbose=1, 
    save_best_only=True,
    save_weights_only=True,
    save_freq='epoch'
)

In [11]:
model = ks.models.Sequential([
    layers.Conv2D(12, (32, 32), activation='relu', input_shape=(300, 300, 3), name='Conv2D_layer1'),
    layers.MaxPool2D((2, 2), name='MaxPooling2D_layer1'),
    layers.Conv2D(36, (16, 16), activation='relu', input_shape=(300, 300, 3), name='Conv2D_layer2'),
    layers.MaxPool2D((4, 4), name='MaxPooling2D_layer2'),
    layers.Flatten(name='Flatten'),
    layers.Dropout(0.15),
    layers.Dense(1000, activation='relu', name='Dense_layer1'),
    layers.Dropout(0.15),
    layers.Dense(100, activation='relu', name='Dense_layer2'),
    layers.Dropout(0.15),
    layers.Dense(3, activation='softmax', name='Output_layer')
])

model.compile(loss=config['loss_function'], optimizer=config['optimizer'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 269, 269, 12)      36876     
_________________________________________________________________
MaxPooling2D_layer1 (MaxPool (None, 134, 134, 12)      0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 119, 119, 36)      110628    
_________________________________________________________________
MaxPooling2D_layer2 (MaxPool (None, 29, 29, 36)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 30276)             0         
_________________________________________________________________
dropout (Dropout)            (None, 30276)             0         
_________________________________________________________________
Dense_layer1 (Dense)         (None, 1000)              3

In [ ]:
model.fit(train_ds, 
          validation_data=valid_ds, 
          epochs=config['epochs'],
         callbacks=[recorder, checkpoint_callback])

Epoch 1/3
  11/1125 [..............................] - ETA: 5:30:11 - loss: 416.4815